# Lex-Minimizer #
In this notebook, we will demonstrate the use of functions that computes the inf-minimizer (`CompInfMin`) and lex-minimizer (`CompLexMin`), as well as other experiments. You can find all source code in `lex.jl`.

In [3]:
pwd()

"/home/xshi0x63/Desktop/YINS/Laplacians.jl/notebooks"

In [47]:
push!(LOAD_PATH,"../src")
using Laplacians

In [48]:
using PyPlot

In [177]:
include("../src/lex.jl") # for development

ModDijkstra (generic function with 1 method)

## Simple examples ##
### Path Graphs ###
A path graph with `n` vertices. Let vertex `1` and `n` be terminals, with voltages `0` and `1` respectively.

In [156]:
# Set up
n = 40
Pn = pathGraph(n)

isTerm = zeros(Bool, n)
isTerm[1] = true
isTerm[n] = true

initVal = zeros(n)
initVal[n] = 1.0


1.0